In [1]:
# Code For Building Hybrid Classification Tree with 2-Means clustering in each Decision Node and Majority Voting in the Leaf Nodes
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans

In [8]:
class C_Node:

    def __init__(self):
        self.m_NodeIndx=-1
        self.m_Impurity=-1
        self.m_NodeDepth=-1
        self.m_ParentNodeIndx=-1
        self.m_LeftChildIndx=-1
        self.m_RightChildIndx=-1
        self.m_IsDecisionNode=None
        self.m_Label=-1
        self.m_Centroids=None
        self.m_LeftVar = None
        self.m_RightVar = None
        self.m_DataLength=None
        
    def setNode(self,nodeIndx,nodeDepth,parentNodeIndx):  
        self.m_NodeIndx = nodeIndx
        self.m_NodeDepth = nodeDepth
        self.m_ParentNodeIndx = parentNodeIndx

class C_Tree:
    
    def __init__(self,maxDepth,maxNodeNum,path,dataNumThresh,impThresh,ImpDropThresh,method):
        self.m_MaxDepth = maxDepth
        self.m_MaxNodeNum = maxNodeNum
        self.m_CurrNodeNum = 0 
        self.m_NodeArray = [C_Node() for i in range(self.m_MaxNodeNum)]
        self.m_Path = path
        self.m_DataNumThresh = dataNumThresh
        self.m_ImpThresh = impThresh
        self.m_ImpDropThresh = ImpDropThresh
        self.m_Method = method
        
    def getImpurity(self,dataFileName):
        datalist = np.genfromtxt(dataFileName, delimiter=',')
        if (len(datalist.shape) == 1):
            return 0
        else:
            y=np.array([int(i) for i in datalist[:,-1]])
            label, label_count = np.unique(y, return_counts=True)
            label_prob = label_count/np.sum(label_count,dtype=np.float64)

            if self.m_Method==1:                                       ##Mis-classification Impurity
                if len(label_prob)==1:
                    imp = 0
                else:
                    imp = 1 - max(label_prob)

            if self.m_Method==2:                                        ##Gini Impurity
                imp = 1 - (np.sum(label_prob**2)) 

            if self.m_Method==3:                                        ##Entropy Impurity
                if (len(label_count)==0) :
                    imp = 0

                else:
                    imp = -1 * np.sum(np.array([p*np.log(p) for p in label_prob]))
            return(imp)
    
    def informationDrop(self,data_left,data_right):    
        filename1 = "LeftFile.csv"
        filename2 = "RightFile.csv"
        np.savetxt(filename1, data_left, delimiter = ",")
        np.savetxt(filename2, data_right, delimiter = ",")
        imp_left = self.getImpurity(filename1)
        imp_right = self.getImpurity(filename2)
        l = len(data_left)
        r = len(data_right)
        totalImp = (l*imp_left + r*imp_right)/(l+r)
        return totalImp

    def twoMeans(self,dataFileName):
        datalist = np.genfromtxt(dataFileName, delimiter=',')
        X = datalist[:,:datalist.shape[1]-1]
        y = np.array([int(i) for i in datalist[:,-1]])
        label, label_count = np.unique(y, return_counts=True)

        kmeans = KMeans(n_clusters=2).fit(X)
        centroids = kmeans.cluster_centers_
        cluster_labels = kmeans.labels_

        data_left=[]
        data_right=[]
        label, label_count = np.unique(cluster_labels, return_counts=True)
        for i in range(len(X)):
            if cluster_labels[i]==label[0]:
                data_left.append(datalist[i])
            if cluster_labels[i]==label[1]:
                data_right.append(datalist[i])     
        data_left=np.array(data_left)
        data_right=np.array(data_right)
        return data_left,data_right,centroids

    def decisionRule(self,x,node):
        mean1 = node.m_Centroids[0]
        mean2 = node.m_Centroids[1]
        dist1 = np.linalg.norm(x-mean1)
        dist2 = np.linalg.norm(x-mean2)
        if dist1 <= dist2:
            return 0
        else:
            return 1
    
    def splitDataFile(self,node_obj,data_left,data_right):  
        filename1 = self.m_Path+"/"+"d_"+str(node_obj.m_LeftChildIndx)+".csv"
        filename2 = self.m_Path+"/"+"d_"+str(node_obj.m_RightChildIndx)+".csv"
        os.makedirs(os.path.dirname(filename1), exist_ok=True)
        os.makedirs(os.path.dirname(filename2), exist_ok=True)
        np.savetxt(filename1, data_left, delimiter = ",")
        np.savetxt(filename2, data_right, delimiter = ",")                                               
    
    def checkTerminationCondition(self,node,datafilename):
        datalist = np.genfromtxt(datafilename, delimiter=',')
        if len(datalist.shape) == 1:
            IsDecisionNode = False
            dataLength = 1
            Label = datalist[-1]
            imp = 0
        else:    
            dataLength = datalist.shape[0]
            X = datalist[:,:-1]
            y = datalist[:,-1]
            label,label_count = np.unique(y,return_counts=True)
            imp = self.getImpurity(datafilename)

            data_left,data_right,centroids = self.twoMeans(datafilename)

            InformationGain = imp - self.informationDrop(data_left,data_right)
            if (dataLength<=self.m_DataNumThresh or imp<=self.m_ImpThresh or node.m_NodeDepth >= self.m_MaxDepth
                or InformationGain <= self.m_ImpDropThresh):        

                IsDecisionNode=False
                Label = label[np.argmax(label_count)]
            else:
                IsDecisionNode=True
                Label = None

        return IsDecisionNode,dataLength,Label,imp,data_left,data_right,centroids
        
    def fit(self,X_train,y_train):
        train_data = np.hstack((X_train,np.matrix(y_train).T))
        fileName = self.m_Path+"/"+"d_0.csv"
        train_data = pd.DataFrame(train_data)
        train_data.to_csv(fileName,index=False,header=False )
        
        self.m_NodeArray[0].setNode(0,0,-1)   # Setting Root Node of the Tree
        self.m_CurrNodeNum = self.m_CurrNodeNum+1

        for nodeCount in range(self.m_MaxNodeNum): 

            if (self.m_NodeArray[nodeCount].m_NodeIndx==nodeCount and 
                self.m_NodeArray[nodeCount].m_LeftChildIndx==-1 and 
                self.m_NodeArray[nodeCount].m_RightChildIndx==-1 and 
                self.m_NodeArray[nodeCount].m_NodeDepth>=0):

                    dataFileName = self.m_Path+"/"+"d_"+str(self.m_NodeArray[nodeCount].m_NodeIndx)+".csv" 

                    isDecisionNode,dataPointNum,label,impurity,data_left,data_right,centroids = self.checkTerminationCondition(
                        self.m_NodeArray[nodeCount],dataFileName)

                    self.m_NodeArray[nodeCount].m_DataLength = dataPointNum
                    self.m_NodeArray[nodeCount].m_Impurity = impurity
                    self.m_NodeArray[nodeCount].m_Label = label
                    self.m_NodeArray[nodeCount].m_Centroids = centroids
                    self.m_NodeArray[nodeCount].m_LeftVar = np.var(data_left[:,:-1],axis=0)
                    self.m_NodeArray[nodeCount].m_RightVar = np.var(data_right[:,:-1],axis=0)
                    if isDecisionNode==False:
                        self.m_NodeArray[nodeCount].m_IsDecisionNode=False
                        print(nodeCount)
                        print("label-"+str(self.m_NodeArray[nodeCount].m_Label))
                        print("depth-"+str(self.m_NodeArray[nodeCount].m_NodeDepth))
                        print("point_count-"+str(dataPointNum))
                        print("impurity-"+str(impurity))
                        print("----------------------")

                    if isDecisionNode==True:
                        self.m_NodeArray[nodeCount].m_IsDecisionNode=True
                        self.m_NodeArray[nodeCount].m_LeftChildIndx=self.m_CurrNodeNum
                        self.m_NodeArray[nodeCount].m_RightChildIndx=self.m_CurrNodeNum+1
                        lci = self.m_CurrNodeNum
                        rci = self.m_CurrNodeNum+1
                        print(str(self.m_NodeArray[nodeCount].m_NodeIndx)+"-----node index")
                        print(str(self.m_NodeArray[nodeCount].m_Label)+"-------node Label")
                        print(str(self.m_NodeArray[nodeCount].m_NodeDepth)+"------node Depth")
                        print(str(dataPointNum)+"----- no. of datapoints")
                        print(str(impurity)+"-------- Impurity")
                        print("--------------")
                        self.m_NodeArray[lci].setNode(lci,self.m_NodeArray[nodeCount].m_NodeDepth+1,
                                self.m_NodeArray[nodeCount].m_NodeIndx)

                        self.m_NodeArray[rci].setNode(rci,self.m_NodeArray[nodeCount].m_NodeDepth+1,
                                self.m_NodeArray[nodeCount].m_NodeIndx)

                        self.splitDataFile(self.m_NodeArray[nodeCount],data_left,data_right)

                        self.m_CurrNodeNum = self.m_CurrNodeNum+2

            else:
                print("Tree Model Trained!!!!!!!!")
                break  

    def predict(self,X_test):
        pred = np.zeros((X_test.shape[0],1))
        leaf_node_ls = []
        for i in range(X_test.shape[0]):
            nodeCount=0
            x = X_test[i]
            while(nodeCount < self.m_MaxNodeNum and self.m_NodeArray[nodeCount].m_IsDecisionNode == True):

                if self.decisionRule(x,self.m_NodeArray[nodeCount])== 0:
                    nodeCount = self.m_NodeArray[nodeCount].m_LeftChildIndx
                else:
                    nodeCount = self.m_NodeArray[nodeCount].m_RightChildIndx
            y_pred = self.m_NodeArray[nodeCount].m_Label
            pred[i][0] = y_pred
            leaf_node_ls.append(nodeCount)
        return(pred,leaf_node_ls)

##################################################################################################################
dataNumThresh = 35
impThresh  = 0.01
impDropThresh = 1e-5
depth = 5
method = 3
path = os.getcwd()

from sklearn.datasets import load_breast_cancer
data = load_breast_cancer(return_X_y=True)
X = data[0]
y = data[1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) 

clf = C_Tree(depth,2**(depth+1)-1,path,dataNumThresh,impThresh,impDropThresh,method)
clf.fit(X_train,y_train)

y_pred,leaf_node_ls = clf.predict(X_test)

0-----node index
None-------node Label
0------node Depth
455----- no. of datapoints
0.6536706828848562-------- Impurity
--------------
1-----node index
None-------node Label
1------node Depth
100----- no. of datapoints
0.056001534354847345-------- Impurity
--------------
2-----node index
None-------node Label
1------node Depth
355----- no. of datapoints
0.4760596480456492-------- Impurity
--------------
3-----node index
None-------node Label
2------node Depth
86----- no. of datapoints
0.06335477530377637-------- Impurity
--------------
4
label-0.0
depth-2
point_count-14
impurity--0.0
----------------------
5-----node index
None-------node Label
2------node Depth
217----- no. of datapoints
0.07291715040368289-------- Impurity
--------------
6-----node index
None-------node Label
2------node Depth
138----- no. of datapoints
0.6879923392546189-------- Impurity
--------------
7-----node index
None-------node Label
3------node Depth
48----- no. of datapoints
0.10126481756679193-------- Impu

In [9]:
print("Two-Means Tree Accuracy: ",accuracy_score(y_test,y_pred))

Two-Means Tree Accuracy:  0.9035087719298246


---
## <center>Kernal-based Classification

In [10]:
def GaussianKernel(X_arr,X,X_central):
    sig_arr = np.std(X_arr,axis=0)
    sig_arr = np.ravel(sig_arr)
    dist_sum = 0
    for i in range(len(X)):
        d1 = (X[i]-X_central[i])/sig_arr[i]
        dist_sum += d1**2
    kernel_val = np.exp(-0.5*dist_sum)
    return(kernel_val)

In [11]:
def kernelbasedClassifier(X_test,leaf_node_ls):
    y_pred = []
    for i in range(X_test.shape[0]):
        leaf_node_idx = leaf_node_ls[i]
        node_file_name = "d_"+str(leaf_node_idx)+".csv"
        node_data_arr = np.genfromtxt(node_file_name,delimiter=",")
        X_node = node_data_arr[:,:-1]
        y_node = node_data_arr[:,-1]
        X = np.ravel(X_test[i,:])
        sum_1 = 0
        sum_2 = 0
        for j in range(X_node.shape[0]):
            X_central = np.ravel(X_node[j,:])
            k = GaussianKernel(X_node,X,X_central)
            sum_1 += k*y_node[j]
            sum_2 += k
        val = sum_1/sum_2
        if val>0.5:
            y_pred.append(1)
        if val<=0.5:
            y_pred.append(0)
    return(y_pred)

In [12]:
y_pred = kernelbasedClassifier(X_test,leaf_node_ls)
print(y_pred)

[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0]


In [13]:
print("Kernal Based Classification Accuracy: ",accuracy_score(y_test,y_pred))

Kernal Based Classification Accuracy:  0.956140350877193
